In [1]:
import sys

from data import crop_upper_part, BinaryDataset
from model import SqueezeModel
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from sklearn import metrics
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import os
from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
ANNOTATED_TEST_DATASET = '/home/filipgulan/Downloads/set/test_set'
CPU_CORES = 8
BATCH_SIZE = 32
USE_GPU = True

In [3]:
# THIS REQUIRES THE master branch of the `torchivsion` package
def data_transformations(input_shape, crop_perc = 0.5):
    return transforms.Compose([
        transforms.Lambda(lambda x: crop_upper_part(np.array(x), crop_perc)),
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.Resize((299, 299)),
        transforms.ToTensor(),
    ])

def var(tensor, volatile=False):
    if USE_GPU:
        tensor = tensor.cuda(0)
    return Variable(tensor, volatile=volatile)

def test_sigmoid(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.sigmoid(logit).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)


def test_softmax(model, test_loader):
    model.eval()
    predictions, ground_truths = [], []
    for i, test_batch in enumerate(test_loader):
        test_x, test_y = (var(test_batch[0], volatile=True),
                          var(test_batch[1], volatile=True))
        logit = model(test_x)
        
        softmax = F.softmax(logit, dim=0).cpu().data.numpy()
        ground_truths.extend(test_y.cpu().data.numpy())
        
        for act in softmax:
            predictions.append(act)
       
        print('Predicting batch {}/{}'.format(i + 1, len(test_loader)), end="\r", flush=True)
        
    return np.array(predictions), np.array(ground_truths)
    

In [5]:
# Load sigmoid model
NUM_CLASSES = 2
MODEL_PATH = "./outdir/CenterLossAvg_epoch_18-valLoss_0.01228-valAcc_0.99669"

model_sigmoid = SqueezeModel(fine_tune=True, num_classes=NUM_CLASSES)

model_sigmoid.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)
model_sigmoid.eval()

if USE_GPU:
    model_sigmoid.cuda(0)

Loaded model from: ./outdir/CenterLossAvg_epoch_18-valLoss_0.01228-valAcc_0.99669


/home/gulan_filip/mozgalo-mb/src/pytorch/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/home/gulan_filip/mozgalo-mb/src/pytorch/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [6]:
# Load softmax model
NUM_CLASSES = 25
MODEL_PATH = "./outdir/25_class_model"

model_softmax = SqueezeModel(fine_tune=True, num_classes=NUM_CLASSES)

model_softmax.load_state_dict(torch.load(MODEL_PATH, map_location=lambda storage, loc: storage))
print("Loaded model from:", MODEL_PATH)
model_softmax.eval()

if USE_GPU:
    model_softmax.cuda(0)

Loaded model from: ./outdir/25_class_model


/home/gulan_filip/mozgalo-mb/src/pytorch/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/home/gulan_filip/mozgalo-mb/src/pytorch/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [7]:
test_transform = data_transformations((3, 370, 400))

In [10]:
test_dataset_binary = BinaryDataset(images_dir=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader_binary = torch.utils.data.DataLoader(test_dataset_binary,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    num_workers=CPU_CORES)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/filipgulan/college/test/Other'

In [6]:
test_dataset_softmax = datasets.ImageFolder(root=ANNOTATED_TEST_DATASET, transform=test_transform)
test_dataset_loader_softmax = torch.utils.data.DataLoader(test_dataset_softmax,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=False,
                                                    num_workers=CPU_CORES)

FileNotFoundError: [Errno 2] No such file or directory: '/home/filipgulan/Downloads/set/test_set'

## Other vs. All

In [13]:
pred, true = test_sigmoid(model_sigmoid, test_dataset_loader_binary)

In [28]:
thr = 0.5
sigmoid_classes = np.copy(pred)
sigmoid_classes[pred > thr] = 1
sigmoid_classes[pred <= thr] = 0

report = metrics.classification_report(true, sigmoid_classes)
print(report)

             precision    recall  f1-score   support

        0.0       0.95      0.99      0.97      5009
        1.0       0.99      0.95      0.97      4991

avg / total       0.97      0.97      0.97     10000



## 25 class softmax

In [15]:
pred, true = test_softmax(model_softmax, test_dataset_loader_softmax)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


TypeError: softmax(): argument 'input' (position 1) must be Tensor, not tuple

In [ ]:
correct = np.argmax(pred)
report = metrics.classification_report(correct, true)
print(report)

# Exporting results for upload

In [8]:
# Class indices when OTher is present
labels = {'Albertsons': 0,
 'BJs': 1,
 'CVSPharmacy': 2,
 'Costco': 3,
 'FredMeyer': 4,
 'Frys': 5,
 'HEB': 6,
 'HarrisTeeter': 7,
 'HyVee': 8,
 'JewelOsco': 9,
 'KingSoopers': 10,
 'Kroger': 11,
 'Meijer': 12,
 'Publix': 13,
 'Safeway': 14,
 'SamsClub': 15,
 'ShopRite': 16,
 'Smiths': 17,
 'StopShop': 18,
 'Target': 19,
 'Walgreens': 20,
 'Walmart': 21,
 'Wegmans': 22,
 'WholeFoodsMarket': 23,
 'WinCoFoods': 24}
class_dict = {v: k for k, v in labels.items()}
class_dict

{0: 'Albertsons',
 1: 'BJs',
 2: 'CVSPharmacy',
 3: 'Costco',
 4: 'FredMeyer',
 5: 'Frys',
 6: 'HEB',
 7: 'HarrisTeeter',
 8: 'HyVee',
 9: 'JewelOsco',
 10: 'KingSoopers',
 11: 'Kroger',
 12: 'Meijer',
 13: 'Publix',
 14: 'Safeway',
 15: 'SamsClub',
 16: 'ShopRite',
 17: 'Smiths',
 18: 'StopShop',
 19: 'Target',
 20: 'Walgreens',
 21: 'Walmart',
 22: 'Wegmans',
 23: 'WholeFoodsMarket',
 24: 'WinCoFoods'}

In [10]:
torch.set_num_threads(8)
IMAGES_FOLDER = '/home/gulan_filip/Downloads/set/test_set'
CSV_PATH = "./results.csv"

# TODO Add batching
results = []
images = sorted([(x, int(x.split(".")[0])) for x in os.listdir(IMAGES_FOLDER)], key=lambda x: x[1])

for ind, (img, _) in enumerate(images):
    with open(os.path.join(IMAGES_FOLDER, img), 'rb') as f:
        image = Image.open(f).convert("RGB")
        
    # make example a torch tensor
    value = test_transform(image)

    test_value = value.unsqueeze(0)
    test_value = Variable(test_value)
    if USE_GPU:
        test_value = test_value.cuda(0)
    

    
    prediction = model_sigmoid(test_value)[0]
    prediction = F.sigmoid(prediction)

    # get the result out and reshape it
    sigmoid = prediction.cpu().data.numpy().flatten()
    if sigmoid[0] > 0.5:
        name = 'Other'
    else:
        prediction = model_softmax(test_value)[0].max(1)[1].cpu().data.numpy()[0]
        # Get names dicitonary
        name = class_dict[prediction]
    results.append(name)
        
#     output = model_softmax(test_value)[0]
#     output = F.softmax(output)
#     if output.max().cpu().data.numpy() < 0.99:
#         name = 'Other'
#     else:      
#         prediction = output.max(1)[1].cpu().data.numpy()[0]
#         # Get names dicitonary
#         name = class_dict[prediction]
#     results.append(name)

        
    if ind % 10 == 0:
        print("Processed {}/{}".format(ind, len(images)))
            

Processed 0/10000
Processed 10/10000
Processed 20/10000
Processed 30/10000
Processed 40/10000
Processed 50/10000
Processed 60/10000
Processed 70/10000
Processed 80/10000
Processed 90/10000
Processed 100/10000
Processed 110/10000
Processed 120/10000
Processed 130/10000
Processed 140/10000
Processed 150/10000
Processed 160/10000
Processed 170/10000
Processed 180/10000
Processed 190/10000
Processed 200/10000
Processed 210/10000
Processed 220/10000
Processed 230/10000
Processed 240/10000
Processed 250/10000
Processed 260/10000
Processed 270/10000
Processed 280/10000
Processed 290/10000
Processed 300/10000
Processed 310/10000
Processed 320/10000
Processed 330/10000
Processed 340/10000
Processed 350/10000
Processed 360/10000
Processed 370/10000
Processed 380/10000
Processed 390/10000
Processed 400/10000
Processed 410/10000
Processed 420/10000
Processed 430/10000
Processed 440/10000
Processed 450/10000
Processed 460/10000
Processed 470/10000
Processed 480/10000
Processed 490/10000
Processed 5

Processed 3960/10000
Processed 3970/10000
Processed 3980/10000
Processed 3990/10000
Processed 4000/10000
Processed 4010/10000
Processed 4020/10000
Processed 4030/10000
Processed 4040/10000
Processed 4050/10000
Processed 4060/10000
Processed 4070/10000
Processed 4080/10000
Processed 4090/10000
Processed 4100/10000
Processed 4110/10000
Processed 4120/10000
Processed 4130/10000
Processed 4140/10000
Processed 4150/10000
Processed 4160/10000
Processed 4170/10000
Processed 4180/10000
Processed 4190/10000
Processed 4200/10000
Processed 4210/10000
Processed 4220/10000
Processed 4230/10000
Processed 4240/10000
Processed 4250/10000
Processed 4260/10000
Processed 4270/10000
Processed 4280/10000
Processed 4290/10000
Processed 4300/10000
Processed 4310/10000
Processed 4320/10000
Processed 4330/10000
Processed 4340/10000
Processed 4350/10000
Processed 4360/10000
Processed 4370/10000
Processed 4380/10000
Processed 4390/10000
Processed 4400/10000
Processed 4410/10000
Processed 4420/10000
Processed 443

Processed 7870/10000
Processed 7880/10000
Processed 7890/10000
Processed 7900/10000
Processed 7910/10000
Processed 7920/10000
Processed 7930/10000
Processed 7940/10000
Processed 7950/10000
Processed 7960/10000
Processed 7970/10000
Processed 7980/10000
Processed 7990/10000
Processed 8000/10000
Processed 8010/10000
Processed 8020/10000
Processed 8030/10000
Processed 8040/10000
Processed 8050/10000
Processed 8060/10000
Processed 8070/10000
Processed 8080/10000
Processed 8090/10000
Processed 8100/10000
Processed 8110/10000
Processed 8120/10000
Processed 8130/10000
Processed 8140/10000
Processed 8150/10000
Processed 8160/10000
Processed 8170/10000
Processed 8180/10000
Processed 8190/10000
Processed 8200/10000
Processed 8210/10000
Processed 8220/10000
Processed 8230/10000
Processed 8240/10000
Processed 8250/10000
Processed 8260/10000
Processed 8270/10000
Processed 8280/10000
Processed 8290/10000
Processed 8300/10000
Processed 8310/10000
Processed 8320/10000
Processed 8330/10000
Processed 834

In [11]:
results[:10]

['Smiths',
 'Target',
 'Other',
 'Target',
 'ShopRite',
 'Walgreens',
 'HarrisTeeter',
 'Other',
 'WinCoFoods',
 'Target']

In [13]:
# Save to csv
with open(CSV_PATH, "w") as f:
    for i, line in enumerate(results):
        f.write(line)
        if i < len(results) - 1:
            f.write("\n")